In [1]:
# import libraries

'''
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer
'''

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# These are all the imports needed for the assignment
%matplotlib inline

# Import nltk package (Natural Language Toolkit)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')

# scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [2]:
import pyspark

In [3]:
import dask.dataframe as dd

In [4]:
# Download the NLTK English tokenizer and the stopwords of all languages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\19495\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\19495\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

client = boto3.client(
    's3',
    aws_access_key_id = 'sukiK@2002',
    aws_secret_access_key = 'sukik',
    region_name = 'ap-west-1'
)

In [5]:
files = ['amazon_reviews_us_Electronics_v1_00.tsv', \
         'amazon_reviews_us_Gift_Card_v1_00.tsv', \
         'amazon_reviews_us_Major_Appliances_v1_00.tsv', \
         'amazon_reviews_us_Office_Products_v1_00.tsv', \
         'amazon_reviews_us_Shoes_v1_00.tsv', \
         'amazon_reviews_us_Toys_v1_00.tsv', \
         'amazon_reviews_us_Watches_v1_00.tsv']

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [7]:
columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', \
           'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
elec_df = pd.read_csv("C:/Users/19495/DS3ProjectFiles/amazon_reviews_us_Electronics_v1_00.tsv.gz", names = columns, sep = '\t').iloc[1:,:]
# 'fars_data/' + files[0]
elec_df = elec_df.sample(n = 1_000_000)
# 200000

columns = ['marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase', 'review_headline', 'review_body', 'review_date']
len(columns)

one_file = pd.read_csv('fars_data/' + files[0], names = columns, sep = '\t').iloc[1:,:]
one_file.head()

one_file.shape[0]+1

In [8]:
one_file = elec_df

## Data Sampling

In [9]:
def df_sampling(df):
    # Since we know that there are more unverified than verified --> we sample based on that
    
    # Since there are no data values in 'verified_purchase' columns that deviate from 'Y' or 'N' we proceed
    verified_count_df = df[df['verified_purchase'] == 'Y']
    unverified_count_df = df[df['verified_purchase'] == 'N']
    
    print("Number of verified purchases:", len(verified_count_df))
    print("Number of unverified purchases:", len(unverified_count_df))
    
    sample_len = len(unverified_count_df)
    
    verified_sample_df = verified_count_df.sample(n = sample_len)
    unified_df = pd.concat([unverified_count_df, verified_sample_df])
    
    print("Number of verified purchases (balanced dataset):", len(unified_df[unified_df['verified_purchase'] == 'Y']))
    print("Number of unverified purchases (balanced dataset):", len(unified_df[unified_df['verified_purchase'] == 'N']))
    
    return unified_df

#test.rename(columns=test.iloc[0])
training_data = pd.DataFrame() #initialize an empty dataframe
testing_data = pd.DataFrame()

one_file = pd.read_csv(data_location + files[1], names = columns, sep = '\t').iloc[1:,:]

In [10]:
one_file.shape

(1000000, 15)

In [11]:
balanced_elec = df_sampling(one_file)
display(balanced_elec.head())

Number of verified purchases: 840248
Number of unverified purchases: 159752
Number of verified purchases (balanced dataset): 159752
Number of unverified purchases (balanced dataset): 159752


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
3085200,US,49456999,RCKQ2M05RVJO7,B00003CWED,840980079,Monster Cable MV2R-2M Monster Video 2 High Resolution Video Cable with RCA Connectors (6.56 ft.) (Discontinued by Manufacturer),Electronics,3,3,4,N,N,Pretty good cable..,"I've recently tried out the Monster Video 2 RCA connectors in my home theater system. They seem to carry more than enough bandwidth for VCRs, and pretty good quality on DVD. I noticed a slight bit of noise when using Monster Video 2, but so fine that its not incredibly noticeable on a 25 inch tv. Other than that, it seems to work fine. Good cable.",2001-02-20
1366359,US,38217713,R3RNB6IXUJL14F,B005HJ94SS,890447288,Dish Network VQ2500 Tailgater Portable Satellite Antenna,Electronics,1,1,1,N,N,"Two years no problems then it stopped working, If you are mechanical Kingcontrol.com will sell you replacement parts",Thank goodness my husband can repair things because there is not a lot of support for tailgater owners. Suggest you buy another product or be prepared to replace if it fails.,2014-05-27
2280732,US,28464094,R33F1A7DDO60Z,B007O5ATGO,125511251,Pioneer Network Ready AV Receiver,Electronics,4,9,13,N,N,"Good Receiver, Bad Pioneer Service and Description","Bought the 1021 last year and wanted to add another system in another room. Purchased this model because the Pioneer site said it was \\""wireless ready\\"" whereas the 1022 was only \\""network ready\\"". Silly us thought that meant this $599 unit could do wireless out of the box similar to our $99 Apple TV. Wrong. When we called Pioneer to explain that more information regarding needing a wireless adapter for this model should be displayed in the comparison window (not 3 layers deep in the specs)we were treated very rudely.<br />We contemplated doing a hardwire but in the end purchased a NETGEAR WNCE2001 after reading about the extremely poor quality and high price of the Pioneer adapter.<br />As far as we could tell this years 1122 seems to be identical to last years 1021, Pioneer just made it seem to be a higher grade model by shifting the model number and price. Both need use an adapter to be wireless.<br />Love the receiver!",2012-10-31
1870347,US,29783028,R4Q45ZGXQX0G3,B00E58X7AS,522491704,Goodies® Replacement screws 4 / tool kit for Monster beats by Dr dre SOLO HD Headphones,Electronics,5,0,0,N,N,nice,"it's hard to find the screw for SOLO HD headphone. although it is such a tiny part, but it is very helpful and solve my problem.",2013-08-03
3033962,US,12157270,R2DPO0QXXZWT8S,B0009RTBY0,973314855,Brit Red By For Women. Eau De Parfum Spray 3.4 Ounces,Electronics,5,2,2,N,N,Brit with a Twist!,"As always, Burberry does not disappoint. If you like the original Burberry Brit perfume, you will love the ever-so-slightly sweeter, Burberry Brit Red. The fragrance lasts and lasts, is warm but not heavy, and is sure to please anyone who prefers a musky scent to a floral. The pleasant packaging also makes this a wonderful gift, especially during the holiday season.",2005-11-10


### Type Conversion and Exploration

In [12]:
def convert_to_int(x):
    return int(float(x))

In [13]:
#convert all mixed datatypes --> string objects (unable to convert to int)
one_file['customer_id'] = one_file['customer_id'].apply(int) #1
one_file['product_parent'] = one_file['product_parent'].apply(int) #4
one_file['star_rating'] = one_file['star_rating'].apply(int) #7
one_file['helpful_votes'] = one_file['helpful_votes'].apply(int) #8
one_file['total_votes'] = one_file['total_votes'].apply(int) #9

one_file.isnull().sum().sort_values(ascending=False)

one_file.dropna(inplace=True) #drop all of the missing values
one_file.isnull().sum().sort_values(ascending=False)

In [14]:
#there may be tabs in the review that is what is leading to the excluded cases when converting to df

'''
train_data['verified_purchase'].value_counts()
test_data['verified_purchase'].value_counts()
'''

one_file.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
414904,US,15355003,RF26FBCE767W7,B0009YJXMI,80062875,Verbatim 4.7GB up to 16x Branded Recordable Disc DVD-R 100-Disc Spindle 95102,Electronics,5,0,0,N,Y,Five Stars,EXCELLENT DEAL. CHEAPER THAN IN STORES.,2015-04-20
2530942,US,14743717,R26DT9UWZV110G,B0016BENZ0,58392094,"C2G/Cables to Go 40448 Value Series Composite Video/Stereo Audio Cable-6 Feet, Black",Electronics,5,0,0,N,Y,simple and great,"This product is easy to use. For the low price, the quality is great. I mainly use it to connect my television to a complete speaker system.",2011-09-21
646435,US,39636308,R1Q1IQW2IMBST,B0092XYLP6,500604734,Formvan Lot 5 Pcs Plastic Battery Holder Case Storage Box For CR123A 18650 16340,Electronics,4,1,1,N,Y,Cheap and works.,Good product to use my AA as D cell in my daughter's toys and swings.,2015-02-13
1431981,US,10908387,R3GA7I7YCIWKNQ,B004LY1LJA,390707571,Digital Silence DS101A Noise Cancelling Earphones / Headphones with Mic (Black),Electronics,4,0,1,N,Y,great noise canceling...,bought them to take them in a 12 hr flight. no need to charge in the complete trip used them for over 25 hrs now and no need to charge yet. very good. very satisfied. very good noise cancelling.,2014-04-14
3085200,US,49456999,RCKQ2M05RVJO7,B00003CWED,840980079,Monster Cable MV2R-2M Monster Video 2 High Resolution Video Cable with RCA Connectors (6.56 ft.) (Discontinued by Manufacturer),Electronics,3,3,4,N,N,Pretty good cable..,"I've recently tried out the Monster Video 2 RCA connectors in my home theater system. They seem to carry more than enough bandwidth for VCRs, and pretty good quality on DVD. I noticed a slight bit of noise when using Monster Video 2, but so fine that its not incredibly noticeable on a 25 inch tv. Other than that, it seems to work fine. Good cable.",2001-02-20


#to check te datatypes within each column

print(train_data.applymap(type))
data_types = train_data.applymap(type)

In [15]:
#the number of customers that gave multiple reviews
#may be valueable to investigate
one_file.groupby('customer_id').count()['marketplace'].sort_values(ascending = False).value_counts()

1     738205
2      75007
3      17207
4       5825
5       2395
6       1102
7        623
8        372
9        258
10       143
11        96
12        61
13        56
14        36
15        30
17        24
16        22
18        13
19        12
21        11
28         8
24         5
20         5
22         5
25         5
34         5
33         3
35         3
26         3
37         2
23         2
30         2
31         2
41         2
52         1
65         1
62         1
61         1
59         1
56         1
53         1
68         1
51         1
44         1
42         1
36         1
32         1
29         1
27         1
69         1
Name: marketplace, dtype: int64

In [16]:
one_file['star_rating'].value_counts()

5    575949
4    173085
1    116028
3     77036
2     57902
Name: star_rating, dtype: int64

## Data Cleaning

In [17]:
# Import re for text cleaning purposes
import re

import nltk.corpus
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\19495\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\19495\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [18]:
def df_cleaning(df, col):
    # Drop rows with na values
    df.dropna(inplace = True)
    
    new_col_name = 'new_' + col
    
    df[new_col_name] = df[col].copy() 
    
    # Remove unwanted formatting characters
    format_strs = dict.fromkeys(['<br /><br />', '&#34', 'br', '&quot', '<br />'], ' ')
    
    for key in format_strs:
        df[new_col_name] = df[new_col_name].apply(lambda review: review.replace(key, format_strs[key]))
    # removing quotes produces smthg like this --> 'The product has great ;sound; --> we must remove punctuation

    
    # Case normalization (lower case)
    df[new_col_name] = df[new_col_name].str.lower()
    
    remove_dict = {"0": "", "1": "", "2": "", "3": "", "4": "", "5": "", "6": "", "7": "", "8": "", "9": "",
                   "(": "", ")":""}
    for key, val in remove_dict.items():
        df[new_col_name] = df[new_col_name].apply(
            lambda x: x.replace(key, val))
        
    # Remove stopwords
    stop_lst = stopwords.words('english')
    #stop_lst += (["can't", "i'm" "i'd", "i've", "i'll", "that's", "there's", "they're"])
    # ****Do we not have to take stopwords out BEFORE removing punctuation? Otherwise words with punct like “cant” remains there
    df[new_col_name] = df[new_col_name].apply(lambda text_body: " ".join([word for word in text_body.split() if word not in (stop_lst)]))
    
    # Removing Unicode Chars (punctuation, URL, @)
    df[new_col_name] = df[new_col_name].apply(lambda rev: re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", rev))
    
    # Lemmatization
    word_lemmatizer = WordNetLemmatizer()
    df[new_col_name] = df[new_col_name].apply(lambda txt: " ".join([(word_lemmatizer.lemmatize(word)) for word in txt.split()]))
    
    return df

cleaned = df_cleaning(one_file)

In [19]:
cleaned2 = df_cleaning(one_file, 'review_body')

In [20]:
cleaned2.get(['review_body', 'new_review_body']).head()

,review_body,new_review_body
414904,EXCELLENT DEAL. CHEAPER THAN IN STORES.,excellent deal cheaper store
2530942,"This product is easy to use. For the low price, the quality is great. I mainly use it to connect my television to a complete speaker system.",product easy use low price quality great mainly use connect television complete speaker system
646435,Good product to use my AA as D cell in my daughter's toys and swings.,good product use aa cell daughter toy swing
1431981,bought them to take them in a 12 hr flight. no need to charge in the complete trip used them for over 25 hrs now and no need to charge yet. very good. very satisfied. very good noise cancelling.,bought take hr flight need charge complete trip used hr need charge yet good satisfied good noise cancelling
3085200,"I've recently tried out the Monster Video 2 RCA connectors in my home theater system. They seem to carry more than enough bandwidth for VCRs, and pretty good quality on DVD. I noticed a slight bit of noise when using Monster Video 2, but so fine that its not incredibly noticeable on a 25 inch tv. Other than that, it seems to work fine. Good cable.",ive recently tried monster video rca connector home theater system seem carry enough bandwidth vcr pretty good quality dvd noticed slight bit noise using monster video fine incredibly noticeable inch tv that seems work fine good cable


In [21]:
cleaned2 = df_cleaning(one_file, 'review_headline')

In [22]:
cleaned2.get(['review_headline', 'new_review_headline']).head()

,review_headline,new_review_headline
414904,Five Stars,five star
2530942,simple and great,simple great
646435,Cheap and works.,cheap work
1431981,great noise canceling...,great noise canceling
3085200,Pretty good cable..,pretty good cable


In [23]:
cleaned2 = df_cleaning(one_file, 'product_title')

In [24]:
cleaned2.get(['product_title', 'new_product_title']).head()

,product_title,new_product_title
414904,Verbatim 4.7GB up to 16x Branded Recordable Disc DVD-R 100-Disc Spindle 95102,verbatim gb x branded recordable disc dvdr disc spindle
2530942,"C2G/Cables to Go 40448 Value Series Composite Video/Stereo Audio Cable-6 Feet, Black",cgcables go value series composite videostereo audio cable foot black
646435,Formvan Lot 5 Pcs Plastic Battery Holder Case Storage Box For CR123A 18650 16340,formvan lot pc plastic battery holder case storage box cra
1431981,Digital Silence DS101A Noise Cancelling Earphones / Headphones with Mic (Black),digital silence dsa noise cancelling earphone headphone mic black
3085200,Monster Cable MV2R-2M Monster Video 2 High Resolution Video Cable with RCA Connectors (6.56 ft.) (Discontinued by Manufacturer),monster cable mvrm monster video high resolution video cable rca connector ft discontinued manufacturer


In [25]:
cleaned2.iloc[0].get('review_body')

'EXCELLENT DEAL. CHEAPER THAN IN STORES.'

## Vader Sentiment Analysis

In [26]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

def get_sentiment_scores(review):
    """
    create new dataframe with just the proportions for each review
    four columns
    neg_prop, pos_prop, neu_prop, compound_prop and will contain these values
    obtained from the vator sentiment algorithm
    """
    snt = analyser.polarity_scores(review)
    #print(f"{sentence} {str(snt)}")
    return snt

In [27]:
get_sentiment_scores(cleaned2.iloc[0].get('review_body')) #without lemmitization

{'neg': 0.0, 'neu': 0.575, 'pos': 0.425, 'compound': 0.5719}

In [28]:
get_sentiment_scores(cleaned2.iloc[0].get('new_review_body')) #with lemmitization

{'neg': 0.0, 'neu': 0.448, 'pos': 0.552, 'compound': 0.5719}

In [29]:
one_file = cleaned2

In [30]:
one_file['rev_dict'] = one_file['new_review_body'].apply(get_sentiment_scores)

In [31]:
def get_neg(review_dict):
    return review_dict['neg']

def get_neu(review_dict):
    return review_dict['neu']

def get_pos(review_dict):
    return review_dict['pos']

def get_compound(review_dict):
    return review_dict['compound']

def only_compound(x):
    dct = get_sentiment_scores(x)
    return dct['compound']

In [32]:
#get neg prop
one_file['neg_prop'] = one_file['rev_dict'].apply(get_neg)
#get neu prop
one_file['neu_prop'] = one_file['rev_dict'].apply(get_neu)
#get pos prop
one_file['pos_prop'] = one_file['rev_dict'].apply(get_pos)
#get compound prop
one_file['compound_prop'] = one_file['rev_dict'].apply(get_compound)

In [34]:
#save the dataframe as a csv file
#one_file.to_csv('electronics_data_cleaned.csv')

### Sentiment Analysis

#df is all data other than rating 3
df = one_file[one_file['star_rating'] != 3]

df['star_rating'].value_counts()

df['positively_rated'] = np.where(df['star_rating'] > 3, 1, 0)

df['positively_rated'].value_counts()

sns.countplot(df['positively_rated'])

##### On all of the data

train_data, test_data = np.split(one_file.sample(frac=1, random_state=1729), [int(0.7 * len(one_file))])
print(train_data.shape, test_data.shape)

X_train = train_data['review_body']
X_train.iloc[0] #to_frame()

Y_train = train_data['star_rating']

X_test = test_data['review_body']
Y_test = test_data['star_rating']

#### TF-IDF - to get predictions of what star rating will be based on text of review

from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer().fit(X_train)

len(vect.get_feature_names()) #unique words

X_train_vect = vect.transform(X_train)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vect, Y_train)

predictions = model.predict(vect.transform(X_test))

len(predictions)

###### Confusion Matrix - shows probabilities for whether cat data is predicted correctly

In [35]:
def plot_confusion_matrix(cm, target_names,
                          fname, epoch,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True, target=None):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools
    plt.style.use('default')

    # # only true if it weren't normalized:
    # accuracy = np.trace(cm) / float(np.sum(cm))
    # misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    cm[np.isnan(cm)] = 0.0

    fig = plt.figure(figsize=(5, 4))
    ax = plt.axes()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    if target == "rule-based":
        plt.title(title + ' for rule-based PF')
    else:
        plt.title(title + ' for MLPF at epoch ' + str(epoch))

    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.2f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlim(-1, len(target_names))
    plt.ylim(-1, len(target_names))
    plt.xlabel('Predicted label')
    # plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    plt.savefig(fname + '.png')
    plt.savefig(fname + '.pdf')
    #plt.close(fig)

    return fig, ax

### SVM Notes

- **Bag of Words (BoW)**
        - BoW encodes an input sentence as the frequency of each word in the sentence. 
        - In this approach, all words contribute equally to the feature vectors.
- **Term Frequency - Inverse Document Frequency (TF-IDF)**
        - TF-IDF is a measure of how important each term is to a specific document, as compared to an overall corpus. 
        - TF-IDF encodes each word as its frequency in the document of interest, divided by a measure of how common the word is across all documents (the corpus).
        - Using this approach, each word contributes differently to the feature vectors.
        - The assumption behind using TF-IDF is that words that appear commonly everywhere are not that informative about what is specifically interesting about a document of interest, so it is tuned to representing a document in terms of the words it uses that are different from other documents. 

- To compare those 2 methods, we will first apply them on the same dataset to analyse sentiment (how positive or negative a text is). In order to make the comparison fair, an **SVM (support vector machine)** classifier will be used to classify positive reviews and negative reviews.

- SVM is a simple yet powerful and interpretable linear model. To use it as a classifier, we need to have at least 2 splits of the data: training data and test data. The training data is used to tune the weight parameters in the SVM to learn an optimal way to classify the training data. We can then test this trained SVM classifier on the test data, to see how well it works on data that the classifier has not seen before. 

We will now create a CountVectorizer object to transform the text data into vectors with numerical values.

To do so, we will initialize a CountVectorizer object, and name it as vectorizer.

4 arguments to initialize a CountVectorizer:

* analyzer: 'word'
 Specify to analyze data from word-level.
 
* max_features: 2000
 Set a max number of unique words.
 
* tokenizer: word_tokenize
 Set to tokenize the text data by using the word_tokenizer from NLTK .
 
* stop_words: stopwords.words('english')
 Set to remove all stopwords in English. We do this since they generally don't provide useful discriminative information.

## KNN

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [37]:
one_file.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date', 'new_review_body',
       'new_review_headline', 'new_product_title', 'rev_dict', 'neg_prop',
       'neu_prop', 'pos_prop', 'compound_prop'],
      dtype='object')

In [38]:
def convert_to_id(x):
    alphabet = {'a': 1, 'c': 3, 'b': 2, 'e': 5, 'd': 4, 'g': 7, 'f': 6, 'i': 9, 'h': 8, 'k': 11, 'j': 10, 'm': 13, 'l': 12, 'o': 15, 'n': 14, 'q': 17, 'p': 16, 's': 19, 'r': 18, 'u': 21, 't': 20, 'w': 23, 'v': 22, 'y': 25, 'x': 24, 'z': 26}
    out = ''
    for i in x:
        if i.lower() in alphabet.keys():
            out += str(alphabet[i.lower()])
        else:
            out += i
    return out

In [40]:
one_file['product_id_convert'] = one_file.get("product_id").apply(convert_to_id)

In [ ]:
one_file['product_category_convert'] = one_file.get("product_category").apply(convert_to_id)

In [41]:
one_file['prod_title_comp'] = one_file.get("new_product_title").apply(only_compound)

In [42]:
one_file['rev_title_comp'] = one_file.get("new_review_headline").apply(only_compound)

def help_prop(vals):
    if vals[1] == 0:
        return 0.5
    else:
        return vals[0] / vals[1]

In [43]:
def helpful_prop(df):  
    vote_series = pd.Series(df['helpful_votes'] / df['total_votes'])
    # !! All nan values in votes_prop should be changed to zero: this means that 0/0 occurred !!
    vote_series = vote_series.fillna(0)
    return vote_series

In [44]:
#NEED TO THINK OF WHAT WOULD HAPPEN IF THE NUMBER OF TOTAL VOTES WOULD BE 0
one_file['help_prop'] = helpful_prop(one_file)
# one_file.get("helpful_votes") / one_file.get("total_votes")
#one_file['help_prop'] = one_file.get("helpful_votes").apply(help_prop)

In [ ]:
#one_file.iloc[2].to_frame()

In [45]:
def id_for_dictionary(dic):
    if len(dic) == 4:
        ind = list(dic.values()).index(max(list(dic.values())[0:-1])) #remove the compound
    else:
        ind = list(dic.values()).index(max(list(dic.values())))
        
    if ind == 1:
        return 0
    elif ind == 0:
        return -1
    else:
        return 1

In [46]:
def id_for_prop(prop):
    if prop < 0.45:
        return -1
    elif prop > 0.55:
        return 1
    else:
        return 0

In [47]:
one_file['rev_bod_id'] = one_file.get("rev_dict").apply(id_for_dictionary)

In [48]:
one_file['help_prop_id'] = one_file.get("help_prop").apply(id_for_prop)

find percentages for the body -- text and display those in a column -- act as labels for the percent positivity of the review body and header themselves

In [49]:
#imp_col = one_file[['verified_purchase', 'prod_title_comp', 'star_rating', 'rev_title_comp', 'neg_prop', 'neu_prop', 'pos_prop', 'help_prop']]
imp_col = one_file[['verified_purchase', 'prod_title_comp', 'star_rating', 'rev_title_comp', 'rev_bod_id', 'help_prop_id']]

In [50]:
imp_col.dtypes

verified_purchase     object
prod_title_comp      float64
star_rating            int64
rev_title_comp       float64
rev_bod_id             int64
help_prop_id           int64
dtype: object

In [51]:
#need to replace review_body and review_headline with percentages for sentiment -- easier to use for classification
#vine needs to be converted to a yes/no binary column similar to how positively rated was determined
#use original dataset with rating 3 available

#X = imp_col.iloc[:, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]].values
X = imp_col.iloc[:, [1,2,3,4,5]].values #only taking in the categories that will be used as a dataframe
y = imp_col.iloc[:, 0].values

"""
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])
"""

'\nfrom sklearn.preprocessing import LabelEncoder\nle = LabelEncoder()\nX[:,1] = le.fit_transform(X[:,1])\n'

In [52]:
X

array([[ 0.    ,  5.    ,  0.    ,  1.    , -1.    ],
       [ 0.34  ,  5.    ,  0.6249,  0.    , -1.    ],
       [ 0.    ,  4.    ,  0.    ,  0.    ,  1.    ],
       ...,
       [ 0.3612,  1.    ,  0.1779,  0.    ,  1.    ],
       [ 0.296 ,  3.    ,  0.4404,  0.    ,  1.    ],
       [ 0.4019,  5.    ,  0.4404,  1.    , -1.    ]])

#Since our dataset containing character variables we have to encode it using LabelEncoder

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [56]:
# Next, we are doing feature scaling to the training and test set of independent variables for reducing the size to smaller values
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

In [57]:
from sklearn.neighbors import KNeighborsClassifier

#we are using 
#5 neighborhood points are required for classifying a given point -- distance metric is using the minkonowski equation
knn_classifier = KNeighborsClassifier(n_neighbors = 20, metric = 'euclidean', p = 1)
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=1,
                     weights='uniform')

In [58]:
y_pred = knn_classifier.predict(X_test)

In [59]:
#We can evaluate our model using the confusion matrix and accuracy score by comparing the predicted and actual test values

from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
ac = accuracy_score(y_test,y_pred)

In [60]:
print(cm)

[[   964  30980]
 [  1682 166363]]


In [61]:
print(ac)

0.8366810174559601


In [ ]:
#Can see the model performance and add more features accordingly -- 
#would be good if the performance is greater than 85%

### Test on a product review - Need to write a *function* for this for taking in the user input

The features that we are looking at!
* 'prod_title_comp', 
* 'star_rating', 
* 'rev_title_comp', 
* 'rev_bod_id', 
* 'help_prop_id'

In [69]:
review_body = "this is a good review"
product_title = "Sony Headphones"
review_title = 'Love the product!'

star_rating = 5
helpful_votes = 1
total_votes = 1

In [70]:
test = pd.DataFrame()
test['review_body'] = np.array([review_body])
test['review_title'] = np.array([review_title])
test['product_title'] = np.array([product_title])
test

,review_body,review_title,product_title
0,this is a good review,Love the product!,Sony Headphones


In [71]:
out = df_cleaning(test, 'review_body')
out = df_cleaning(out, 'review_title')
out = df_cleaning(out, 'product_title')
out

,review_body,review_title,product_title,new_review_body,new_review_title,new_product_title
0,this is a good review,Love the product!,Sony Headphones,good review,love product,sony headphone


In [72]:
out['review_body'][0]

'this is a good review'

def get_sentiment_proportions(review):
    """
    create new dataframe with just the proportions for each review
    four columns
    neg_prop, pos_prop, neu_prop, compound_prop and will contain these values
    obtained from the vator sentiment algorithm
    """
    snt = analyser.polarity_scores(review)
    #print(f"{sentence} {str(snt)}")
    neg = snt['neg']
    neu = snt['neu']
    pos = snt['pos']
    #compound = snt['compound']
    return neg, neu, pos

neg, neu, pos = get_sentiment_proportions(out.get("new_review_body").iloc[0])

product_category = convert_to_id(product_category)
product_title = only_compound(product_title)
rev_title = only_compound(review_title)

In [73]:
rev_bod_id = id_for_dictionary(analyser.polarity_scores(out['new_review_body'][0]))
help_prop_id = id_for_prop(helpful_votes / total_votes)
prod_title_comp = only_compound(out['new_product_title'][0])
rev_title_comp = only_compound(out['new_review_title'][0])

Predicted: 'verified_purchase'

User Input: 'prod_title_comp', 'product_category_convert', 'star_rating', 'helpful_votes', 'total_votes', 'neg_prop', 'neu_prop', 'pos_prop'
- 8 fields

In [74]:
rev_input_test = np.array([[prod_title_comp, star_rating, rev_title_comp, rev_bod_id, help_prop_id]])
rev_input_test

array([[0.    , 5.    , 0.6369, 1.    , 1.    ]])

In [75]:
prediction, probabilities = knn_classifier.predict(rev_input_test), knn_classifier.predict_proba(rev_input_test)[0]

In [76]:
prediction

array(['Y'], dtype=object)

In [77]:
probabilities

array([0.2, 0.8])

classifier?

In [78]:
def interpret_prediction(review, pred, proba):
    proba = [round(proba[0], 3), round(proba[1], 3)]
    if prediction[0] == 'Y':
        print(f'"{review}" is predicted to be a VERIFIED review, with {proba[1]*100}% probability of being VERIFIED and {proba[0]*100}% probability of being UNVERIFIED')
    if prediction[0] == 'N':
        print(f'"{review}" is predicted to be an UNVERIFIED review, with {proba[0]*100}% probability of being UNVERIFIED and {proba[1]*100}% probability of being VERIFIED')
        
interpret_prediction(review_test, prediction, probabilities)

NameError: name 'review_test' is not defined

In [79]:
from joblib import dump, load

In [80]:
knn_classifier

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=20, p=1,
                     weights='uniform')

In [ ]:
#name = 'knn_working_model.joblib'
#path = 'KNNModelFiles/'
#dump(knn_classifier, path+name)

In [ ]:
#knn_classifier = load(path+name)

In [237]:
name = 'knn_electronics_million_model.joblib'
dump(knn_classifier, name)
knn_classifier = load(name)

## Getting Reviews

In [105]:
# Getting the indices of the reviews that were predicted correctly
ind1 = list((y_test == y_pred) & (y_test == 'N'))#.index(True)
# & (y_test == 'N')
[i for i, x in enumerate(ind1) if x]

[900,
 1141,
 1242,
 1516,
 2021,
 2094,
 2334,
 2382,
 2629,
 2836,
 2927,
 3112,
 3137,
 3419,
 3745,
 3811,
 3881,
 4046,
 4074,
 4163,
 4190,
 4196,
 4402,
 4591,
 4698,
 4802,
 5134,
 6207,
 6545,
 7220,
 7406,
 7604,
 7685,
 8087,
 8692,
 9374,
 9533,
 9628,
 9737,
 10122,
 10384,
 10960,
 11160,
 11395,
 11434,
 11575,
 12321,
 12497,
 12612,
 12730,
 12808,
 12878,
 12948,
 13023,
 13200,
 13259,
 13263,
 13885,
 14073,
 14217,
 14225,
 14725,
 14797,
 15247,
 15321,
 15333,
 15381,
 15533,
 15888,
 15890,
 16087,
 16300,
 16327,
 16340,
 16395,
 16609,
 16863,
 17182,
 17344,
 17350,
 18124,
 18281,
 18394,
 18455,
 19175,
 19469,
 19502,
 19523,
 19677,
 19682,
 19970,
 20594,
 20733,
 21039,
 21249,
 22236,
 22273,
 22576,
 22621,
 22675,
 23184,
 23330,
 23682,
 23749,
 24093,
 24971,
 25325,
 25362,
 25511,
 26262,
 26440,
 26889,
 27009,
 27588,
 27748,
 28140,
 28289,
 28881,
 29059,
 29067,
 29122,
 29370,
 29635,
 30241,
 30280,
 30386,
 30437,
 30598,
 30624,
 30720,


In [222]:
ind1 = 4402
# 3137, 3745
X_test[ind1]

array([0.    , 5.    , 0.9186, 0.    , 0.    ])

In [223]:
test = list(X)
test;

In [224]:
test2 = (test == X_test[ind1])
test2

array([[ True,  True, False, False, False],
       [False,  True, False,  True, False],
       [ True, False, False,  True, False],
       ...,
       [False, False, False,  True, False],
       [False, False, False,  True, False],
       [False,  True, False, False, False]])

In [225]:
ind2 = list(np.all(test2, axis = 1))#.index(True)
ind2
n_lst = [i for i, x in enumerate(ind2) if x]
(n_lst)

[313740, 446576]

In [228]:
one_file.iloc[313740].to_frame().loc[['review_body', 'review_headline', 'product_title', 'star_rating', 'helpful_votes', 'total_votes', 'verified_purchase']]

,3001848
review_body,These are great rechargable batteries. Much easier to use than the type I used to have. They work right away - the charger is very easy to use - and they last a long long time!! Great way to do my little part to help the earth and save myself some $$$.
review_headline,I love an easy way to help the earth!! (and save $)
product_title,Sanyo Eneloop NiMH Battery Charger with 4AA NiMH Rechargable Batteries (Discontinued by Manufacturer)
star_rating,5
helpful_votes,2
total_votes,4
verified_purchase,N


In [229]:
review_body = "These are great rechargable batteries. Much easier to use than the type I used to have. They work right away - the charger is very easy to use - and they last a long long time!! Great way to do my little part to help the earth and save myself some $$$."
review_title = 'I love an easy way to help the earth!! (and save $)'
product_title = 'Sanyo Eneloop NiMH Battery Charger with 4AA NiMH Rechargable Batteries (Discontinued by Manufacturer)'
star_rating = 5
helpful_votes = 2
total_votes = 4

In [230]:
test = pd.DataFrame()
test['review_body'] = np.array([review_body])
test['review_title'] = np.array([review_title])
test['product_title'] = np.array([product_title])
test

,review_body,review_title,product_title
0,These are great rechargable batteries. Much easier to use than the type I used to have. They work right away - the charger is very easy to use - and they last a long long time!! Great way to do my little part to help the earth and save myself some $$$.,I love an easy way to help the earth!! (and save $),Sanyo Eneloop NiMH Battery Charger with 4AA NiMH Rechargable Batteries (Discontinued by Manufacturer)


In [231]:
out = df_cleaning(test, 'review_body')
out = df_cleaning(out, 'review_title')
out = df_cleaning(out, 'product_title')
out

,review_body,review_title,product_title,new_review_body,new_review_title,new_product_title
0,These are great rechargable batteries. Much easier to use than the type I used to have. They work right away - the charger is very easy to use - and they last a long long time!! Great way to do my little part to help the earth and save myself some $$$.,I love an easy way to help the earth!! (and save $),Sanyo Eneloop NiMH Battery Charger with 4AA NiMH Rechargable Batteries (Discontinued by Manufacturer),great rechargable battery much easier use type used have work right away charger easy use last long long time great way little part help earth save,love easy way help earth save,sanyo eneloop nimh battery charger aa nimh rechargable battery discontinued manufacturer


In [232]:
rev_bod_id = id_for_dictionary(analyser.polarity_scores(out['new_review_body'][0]))
prop = 1/2
help_prop_id = id_for_prop(prop)
prod_title_comp = only_compound(out['new_product_title'][0])
print(prod_title_comp)
rev_title_comp = only_compound(out['new_review_title'][0])

0.0


In [233]:
rev_input_test = np.array([[prod_title_comp, star_rating, rev_title_comp, rev_bod_id, help_prop_id]])
rev_input_test

array([[0.    , 5.    , 0.9186, 0.    , 0.    ]])

In [234]:
prediction, probabilities = knn_classifier.predict(rev_input_test), knn_classifier.predict_proba(rev_input_test)[0]

In [235]:
prediction

array(['N'], dtype=object)

In [236]:
probabilities

array([0.5, 0.5])